In [5]:
import torch
import numpy as np

## Naive Bayes Algorithms

In [6]:
def bayes_MAP(X, y):
    theta = []
    y_1 = []
    y_0 = []
    for j in range(X.shape[1]):
        prob_y_1 = torch.sum(y)/len(y)
        prob_y_0 = 1-prob_y_1

        prob_x_y_1 = 0
        prob_x_y_0 = 0
        for i in range(X.shape[0]):
            if (y[i] == 0 and X[i][j] == 1):
                prob_x_y_0 += 1
            if (y[i] == 1 and X[i][j] == 1):
                prob_x_y_1 += 1
        prob_x_y_1 = prob_x_y_1/X.shape[0]
        prob_x_y_0 = prob_x_y_0/X.shape[0]
        prob_1 = prob_x_y_1/prob_y_1
        prob_0 = prob_x_y_0/prob_y_0
        y_1.append(prob_1)
        y_0.append(prob_0)
    theta.append(y_0)
    theta.append(y_1)
    return torch.FloatTensor(theta)

In [7]:
def bayes_MLE(y):
    return 1-torch.sum(y)/len(y)

In [8]:
def bayes_classify(theta, p, X):
    y = []
    for i in range(X.shape[0]):
        y_1 = 0
        y_0 = 0
        for j in range(X.shape[1]):
            a = 0
            b = 0
            if (X[i][j] == 1):
                a = torch.log(theta[0][j])
                b = torch.log(theta[1][j])
            else:
                a = torch.log(1-theta[0][j])
                b = torch.log(1-theta[1][j])
            if(a < -10000):
                a = 0
            if (b < -100):
                b = 0
            y_0 += a
            y_1 += b
        y_0 += torch.log(p)
        y_1 += torch.log((1-p))
        if (y_1 > y_0):
            y.append(1)
        else:
            y.append(0)
    return torch.Tensor(y)

## Accuracy Check

In [9]:
x, y = torch.load("bayes_train.pth")
D = bayes_MAP(x, y)
p = bayes_MLE(y)

xtest, ytest = torch.load("bayes_test.pth")
ypred = bayes_classify(D, p, xtest)

In [13]:
accuracy = 1 - sum(abs(ypred - ytest))/len(ypred)
print(accuracy)

tensor(0.9560)


## Gaussian Naive Bayes Algorithm

In [14]:
def gaussian_MAP(X, y):
    mu = [[],[]]
    sigma = [[],[]]
    for i in range(X.shape[1]):
        # for each column
        mu_1 = torch.sum(torch.Tensor(X[:,i]).flatten()*torch.Tensor(y))
        low_1 = torch.sum(y)
        mu_0 = torch.sum(torch.Tensor(X[:,i]).flatten()*(torch.abs(1-torch.Tensor(y))))
        low_0 = len(y) - low_1
        mu_0 = mu_0/low_0
        mu_1 = mu_1/low_1
        mu[0].append(mu_0)
        mu[1].append(mu_1)
    
    for i in range(X.shape[1]):
        sigma_1 = torch.sum((torch.Tensor(X[:,i]).flatten()-torch.Tensor(mu[1][i]))**2 * torch.Tensor(y))
        low_1 = torch.sum(y)
        sigma_0 = torch.sum((torch.Tensor(X[:,i]).flatten()-torch.Tensor(mu[0][i]))**2 * torch.abs(1-torch.Tensor(y)))
        low_0 = len(y) - low_1
        sigma_1 = sigma_1/low_1
        sigma_0 = sigma_0/low_0
        sigma[0].append(sigma_0)
        sigma[1].append(sigma_1)
    return torch.Tensor(mu), torch.Tensor(sigma)

In [15]:
def gaussian_MLE(y):
    return 1-torch.sum(y)/len(y)

In [16]:
def gaussian_classify(mu, sigma2, p, X):
    y = []
    for j in range(X.shape[0]):
        y_1 = 1
        y_0 = 1
        cur_x = torch.Tensor(X[j])
        y_1 = torch.sum(torch.log((torch.e**((-1/2)*((cur_x-mu[1])/(sigma2[1])**(1/2))**2))/(2*torch.pi*sigma2[1])**(1/2)))
        y_0 = torch.sum(torch.log((torch.e**((-1/2)*((cur_x-mu[0])/(sigma2[0])**(1/2))**2))/(2*torch.pi*sigma2[0])**(1/2)))
        y_1 += torch.log(1-p)
        y_0 += torch.log(p)
        if(y_1 > y_0):
            y.append(1)
        else:
            y.append(0)
    return torch.Tensor(y)

## Accuracy Check

In [17]:
x, y = torch.load("gaussian_train.pth")
D = bayes_MAP(x, y)
p = bayes_MLE(y)

xtest, ytest = torch.load("gaussian_test.pth")
ypred = bayes_classify(D, p, xtest)

In [18]:
accuracy = 1 - sum(abs(ypred - ytest))/len(ypred)
print(accuracy)

tensor(0.9100)
